In [2]:
import pandas as pd
from networkx import to_networkx_graph

sent_df = pd.read_csv('Assignment 3 Sentiment Scores.csv')

review_df = pd.read_excel('Edmunds Reviews.xlsx', sheetname=None)
# review_df
# sent_df.head()
# xls = pd.ExcelFile('Edmunds Reviews.xlsx')
# df1 = xls.parse('Sheet1')
# df2 = xls.parse('Sheet2')
# df3 = xls.parse('Sheet3')
# df4 = xls.parse('Sheet4')
# df5 = xls.parse('Sheet5')
# df6 = xls.parse('Sheet6')

df = pd.concat(review_df, ignore_index=True)

#Make df with those reviews in the 'post' column
df_test = df[df['Post'].notnull()]


del df_test['Review']
df_test['Review'] = df_test['Post']
del df_test['Post']

df = df[df['Post'].isnull()] #Remove those null reviews
df_final = pd.concat([df,df_test], axis=0, ignore_index=True)
del df_final['Post'] # Get rid of useless Post column
# df_final[df_final['Review'].isnull()] # Can see that no reviews are null
df_final.head()
df_final.shape

C:\Python\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1308, 1)

In [3]:
## Clean up the data
def remove_weird_char(string):
    import re
    string = string.lower()
    words = string.split()
    refined = []
    for i in words:
        refined.append(re.sub("[^\w]+", " ", i))
    sentence = ' '.join([word for word in refined])
    return(sentence)

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")
def rmStopWords(text):
        text = ' '.join([word for word in set(text.split()) if word not in cachedStopWords])
        return text
    
#define function to replace some punctuation with spaces
def rep_punct(text):
    return text.lower().replace('.',' ').replace('!',' ').replace(',',' ').replace('?',' ').replace(')',' ').replace('(',' ')

# def replace_brand(text):
#     aset = set(text.split())
#     overlap = aset.intersection(set(brand_df.keys()))
#     aset = (aset-overlap).union(set([brand_df[x] for x in overlap]))
#     return ' '.join([word for word in aset])

In [4]:
# Read the brand df in 
brand_df = pd.read_csv('brand_list.csv') # !!! I uploaded this csv to the github as well !!! 
brand_df = brand_df.set_index('model')['brand'].to_dict()
brand_df.keys()

dict_keys(['a6', 'a8', '3series', '5series', '7series', 'xj', 'es', 'ls', 'rx', 'sclass'])

In [5]:
## Create function to model names into brand+model names  

def replace_brand(text):
    text = text.lower()
    alist = list(text.split())
    aset= set(alist)
    overlap = aset.intersection(set(brand_df.keys()))
    for word in aset: #For the words in the set of words...
        if word in overlap: #If the word is in the brand dict
            new_word = ''
            new_word = brand_df[word]+word #Make a new word that is both the brand and model name
            text = text.replace(word,new_word) #Replace it 
    return text

## Need to use remove_punct to see correct small samples
# print(df_final['Review'][1])
# replace_brand(df_final['Review'][1].lower())


In [6]:
## Apply it all
df_final['Review'] = df_final['Review'].apply(rep_punct).apply(replace_brand).apply(rmStopWords).apply(remove_weird_char)

In [126]:
## Testing out counting the reviews 

# Define positive words (arbitray)
pos_words = ['nice', 'love', 'safe', 'fast', 'value', 'great', 'fantastic', 'good', 'happy', 'control', 'strong']
# Define negative words (arbitrary)
neg_words = ['hate', 'despise', 'crap', 'old', 'shit', 'slow', 'bad', 'expensive', 'dissapointed']
#Create sentiment dictionary
sent_dict = {'audia6':0, 'audia8':0, 'bmw3series':0, 'bmw5series':0, 'bmw7series':0, 'jaguarxj':0, 'lexuses':0, 'lexusls':0, 'lexusrx':0, 'mercedessclass':0}
# Create brand list 
brand_list = ['audia6', 'audia8', 'bmw3series', 'bmw5series', 'bmw7series', 'jaguarxj', 'lexuses', 'lexusls', 'lexusrx', 'mercedessclass']

for i in range(df_final.shape[0]):
    #Clean up the data a bit
    sentence = str(df_final['Review'][i])
    sentence = sentence.lower()
    sentence = sentence.strip()
    words = sentence.split(' ')
    for j in range(len(words)): #look through all the words in review...
        if words[j] in brand_list: #If we find a brand name...
            sent_words = words[j-10:j+10] #Get the words around it 
            for w in range(len(sent_words)): #Look at the words around it
                if sent_words[w] in pos_words: #For every positive word...
                    sent_dict[words[j]] += 1 #Add one 
                if sent_words[w] in neg_words: #For every negative word...
                    sent_dict[words[j]] -= 1 #Subtract one
sent_dict
## Need to output dictionary of 'brand : score'

{'audia6': 9,
 'audia8': 78,
 'bmw3series': 6,
 'bmw5series': 0,
 'bmw7series': 11,
 'jaguarxj': 10,
 'lexuses': 4,
 'lexusls': 29,
 'lexusrx': 30,
 'mercedessclass': 4}

In [119]:
def sentiment_review(review, brands):
    sent_dict = {'audia6':0, 'audia8':0, 'bmw3series':0, 'bmw5series':0, 'bmw7series':0, 'jaguarxj':0, 'lexuses':0, 'lexusls':0, 'lexusrx':0, 'mercedessclass':0}
    #Clean up the data a bit
    sentence = str(review)
    sentence = sentence.lower()
    sentence = sentence.strip()
    words = sentence.split(' ')
    for j in range(len(words)): #look through all the words in review...
        if words[j] in brands: #If we find a brand name...
            sent_words = words[j-10:j+10] #Get the words around it 
            for w in range(len(sent_words)): #Look at the words around it
                if sent_words[w] in pos_words: #For every positive word...
                    sent_dict[words[j]] += 1 #Add one 
                if sent_words[w] in neg_words: #For every negative word...
                    sent_dict[words[j]] -= 1 #Subtract one
    return sent_dict

In [120]:
# Raw look into sentiment_scores function -- just testing things out 
sent_scores = {}
for item in brand_list:
    sent_scores[item] = list()
for i in range(df_final.shape[0]):
    sent_values = sentiment_review(df_final['Review'][i], brand_list)
    for item in brand_list:
        sent_scores[item].append(sent_values[item])
# sum(sent_scores.values())

In [121]:
## Convert all brands into 'ES', 'LS', ect. 
def sentiment_scores(reviews_data, brand_list):
    sent_scores = {}
    for item in brand_list:
        sent_scores[item] = list()
    for review_id in range(0, len(reviews_data)):
        sent_values = sentiment_review(reviews_data['Review'][review_id], brand_list)
        for item in brand_list:
            sent_scores[item].append(sent_values[item])
    fin = pd.DataFrame.from_dict(sent_scores, orient='columns')
    fin.to_csv('Sentiment_Scores.csv', index=False)
    return sent_scores

In [122]:
## Missing sentiment_review code from Aditya

In [123]:
def form_network(edge_list):
    
    import networkx as nx
    import pylab 
    directed_graph = nx.DiGraph()
    directed_graph.add_weighted_edges_from(edge_list, weight = 'weight')
    nx.draw(directed_graph)
    pylab.show()
    
    pagerank_unweighted = nx.pagerank(directed_graph, alpha=0.99, weight=1)
    
    print(pagerank_unweighted)
    print(len(pagerank_unweighted))
    

In [124]:
def create_edges(sent_sco):
    
    edge_list = []
    brand_list = sent_sco.keys()
    for x in range(0, len(brand_list)):
        for y in range(x+1, len(brand_list)):
            positive_sent = []
            negative_sent = []
            for i in range(0, len(sent_sco[brand_list][0])):
                sent_x = sent_sco[brand_list[x]][i]
                sent_y = sent_sco[brand_list[y]][i]

In [125]:
# if __name__ == '__main__':

# reviews_data = pd.read_excel(sdfs, encoding = 'latin1')
reviews_data = df_final
brand_list = ['audia6', 'audia8', 'bmw3series', 'bmw5series', 'bmw7series', 'jaguarxj', 'lexuses', 'lexusls', 'lexusrx', 'mercedessclass']
sentiment_scores_final = sentiment_scores(reviews_data, brand_list)
########################################
##### The create_edges and form_network function are giving me trouble, will worry about them later #####
# edge_list = create_edges(sentiment_scores)
# pagerank = form_network(edge_list)
########################################
sentiment_scores_final

{'audia6': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,